In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("banco_de_imoveis.csv")

In [2]:
df.head()

,id,url,header,address,area,rooms,bathrooms,garages,amenities,price,condo,crawler,crawled_at
0,2472650173,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos à Venda, 64m²","Avenida Portugal, 401 - Brooklin, São Paulo - SP",64,2 Quartos,2 Banheiros,1 Vaga,Elevador\nAr-condicionado\nPlayground\nMóvel p...,R$ 715.000,R$ 778,Brooklin,2021-03-11 01:57
1,https://www.vivareal.com.br/imoveis-lancamento...,https://www.vivareal.com.br/imoveis-lancamento...,Only Cidade Jardim,"Avenida Ulysses Reis de Mattos, 100 - Real Par...",186-252,3 Quartos,4 Banheiros,2-4 Vagas,NaN,A partir de\nR$ 2.429.620,NaN,Brooklin,2021-03-11 01:57
2,2463778941,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos à Venda, 83m²","Rua Nova York, 470 - Brooklin, São Paulo - SP",83,2 Quartos,3 Banheiros,2 Vagas,Piscina\nElevador\nCondomínio fechado\nEspaço ...,R$ 1.350.000,R$ 600,Brooklin,2021-03-11 01:57
3,2505723836,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 3 Quartos à Venda, 75m²","Avenida Portugal, 371 - Brooklin, São Paulo - SP",75,3 Quartos,2 Banheiros,1 Vaga,Academia\nCondomínio fechado,R$ 680.000,R$ 717,Brooklin,2021-03-11 01:57
4,2497648375,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com Quarto à Venda, 47m²","Rua Michigan - Brooklin, São Paulo - SP",47,1 Quarto,2 Banheiros,1 Vaga,Piscina\nVaranda gourmet\nMobiliado\nChurrasqu...,R$ 750.000,R$ 463,Brooklin,2021-03-11 01:57


# Objetivos
- Remover imóveis duplicados
- Remover propagandas
- Colunas numericas devem ter só um número

## Desafios
- Lidar com Amenities
- Extrair bairro do endereço
- Lidar com missing data

# Alguns comandos de pandas utilizados

`df` -> dataframe que estamos trabalhando

`df.head()` -> mostra as 5 primeiras linhas do dataframe

`df.tail()` -> mostra as 5 ultimas linhas do dataframe

`df.columns` -> Lista todas as colunas do dataframe

`df.shape` -> Dimensões do dataframe (numero de linhas e colunas)

`df.duplicated()` -> Procura linhas com TODOS os valores duplicados

`df.duplicated(subset=["id"])` -> procura linhas onde so o ID esta duplicado

`df.drop_duplicates(keep="first")` -> mantém o primeiro da lista de duplicados. Outra opção seria o last.

`w.isnumeric()` -> onde w é uma string, retorna True se w for numérico ou False se não for

`df["NOMEDACOLUNA"].str.split(" ")` -> Separa as strings da coluna NOMEDACOLUNA string no espaco. Ex: "Um brinquedo"-> ["Um", "brinquedo"]

`df["NOMEDACOLUNA"].str.replace("a", "b")` -> Para todas as strings da coluna NOMEDACOLUNA, substitui a por b. Ex: "abacate" -> "bbbcbte"

`df.reset_index(drop=False)` -> Re-constroi o indice sem pular valores. Util sempre que fizer alguma operacao que remova linhas para evitar warnings

`df["NOMEDACOLUNA"].isnull()` -> Filtro para detectar null na coluna NOMEDACOLUNA

`df.dropna()` -> Remove linhas que tiverem nulls.

`df.dropna(subset=["NOMEDACOLUNA"])` -> Remove linhas que tiverem nulls na coluna NOMEDACOLUNA.

`df.fillna("ERRO")` -> Substitui valores null pelo valor "ERRO"
`pd.to_datetime(df_sem_anuncios["crawled_at"], format="%Y-%m-%d %H:%M")` -> Transforma uma coluna de strings em datetime (format: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

`.str.lower()` -> Passa pra minuscula
`.str.replace(" ", "_")` -> substitui " " por "_"
`.str.normalize("NFKD").str.encode("ascii", errors="ignore").decode("utf-8")` - tirar os acentos 

# Removendo duplicados

In [3]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
19435    False
19436    False
19437    False
19438    False
19439    False
Length: 19440, dtype: bool

In [4]:
# Quantidades de linhas únicas que estão repetidas
sum(df.duplicated())

713

In [5]:
# Removendo as repetidas mantendo apenas a primeira de cada repetição
df = df.drop_duplicates(keep="first", subset=[coluna for coluna in df.columns if coluna!="crawled_at"]).reset_index(drop=True)

In [6]:
df[df["id"]=="1037345211"]

,id,url,header,address,area,rooms,bathrooms,garages,amenities,price,condo,crawler,crawled_at
11758,1037345211,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos à Venda, 66m²","Pinheiros, São Paulo - SP",66,2 Quartos,2 Banheiros,1 Vaga,NaN,R$ 858.000,R$ 528,Pinheiros,2021-03-11 03:45


## Remover propagandas

In [7]:
df

,id,url,header,address,area,rooms,bathrooms,garages,amenities,price,condo,crawler,crawled_at
0,2472650173,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos à Venda, 64m²","Avenida Portugal, 401 - Brooklin, São Paulo - SP",64,2 Quartos,2 Banheiros,1 Vaga,Elevador\nAr-condicionado\nPlayground\nMóvel p...,R$ 715.000,R$ 778,Brooklin,2021-03-11 01:57
1,https://www.vivareal.com.br/imoveis-lancamento...,https://www.vivareal.com.br/imoveis-lancamento...,Only Cidade Jardim,"Avenida Ulysses Reis de Mattos, 100 - Real Par...",186-252,3 Quartos,4 Banheiros,2-4 Vagas,NaN,A partir de\nR$ 2.429.620,NaN,Brooklin,2021-03-11 01:57
2,2463778941,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos à Venda, 83m²","Rua Nova York, 470 - Brooklin, São Paulo - SP",83,2 Quartos,3 Banheiros,2 Vagas,Piscina\nElevador\nCondomínio fechado\nEspaço ...,R$ 1.350.000,R$ 600,Brooklin,2021-03-11 01:57
3,2505723836,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 3 Quartos à Venda, 75m²","Avenida Portugal, 371 - Brooklin, São Paulo - SP",75,3 Quartos,2 Banheiros,1 Vaga,Academia\nCondomínio fechado,R$ 680.000,R$ 717,Brooklin,2021-03-11 01:57
4,2497648375,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com Quarto à Venda, 47m²","Rua Michigan - Brooklin, São Paulo - SP",47,1 Quarto,2 Banheiros,1 Vaga,Piscina\nVaranda gourmet\nMobiliado\nChurrasqu...,R$ 750.000,R$ 463,Brooklin,2021-03-11 01:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18209,2511444468,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos à Venda, 56m²","Rua Purus - Tucuruvi, São Paulo - SP",56,2 Quartos,1 Banheiro,1 Vaga,Playground\nSalão de festas\nSalão de jogos,R$ 359.000,NaN,Tucuruvi,2021-03-11 04:46
18210,2500619794,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 3 Quartos à Venda, 78m²","Rua Manuel Gaya, 945 - Tucuruvi, São Paulo - SP",78,3 Quartos,2 Banheiros,2 Vagas,Piscina\nVaranda gourmet\nChurrasqueira\nVaran...,R$ 670.000,R$ 650,Tucuruvi,2021-03-11 04:46
18211,2505112299,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 3 Quartos à Venda, 82m²","Avenida Coronel Sezefredo Fagundes - Tucuruvi,...",82,3 Quartos,2 Banheiros,1 Vaga,Aquecimento,R$ 675.000,R$ 650,Tucuruvi,2021-03-11 04:46
18212,2505098315,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 3 Quartos à Venda, 82m²","Avenida Coronel Sezefredo Fagundes - Tucuruvi,...",82,3 Quartos,2 Banheiros,1 Vaga,Aquecimento,R$ 675.000,R$ 650,Tucuruvi,2021-03-11 04:46


In [8]:
# Cria uma lista com True caso o id seja numérico e False caso não seja
filtro_de_anuncios = [_id.isnumeric() for _id in df["id"]]

In [9]:
# Usamos o filtro para pegar apenas as linhas onde o id é numérico
df = df[filtro_de_anuncios].reset_index(drop=True)

In [10]:
df.shape

(15524, 13)

# Remover strings de campos numericos

In [11]:
# Verificamos os valores únicos
set(df["rooms"])

{'-- Quarto', '1 Quarto', '2 Quartos', '3 Quartos', '4 Quartos', '5 Quartos'}

In [12]:
# Fazemos o split no espaço, pegamos a primeira parte. Depois substituimos o __ por 0.
# Ex: "1 Quartos".split(" ") = [1, Quartos]
# Pegamos a primeira parte: [1]
# Substituimos "--" por 0. Como não tem fica 1 mesmo.
set(df["rooms"].str.split(" ").str[0].str.replace("--","0"))

{'0', '1', '2', '3', '4', '5'}

In [13]:
df["rooms_limpo"] = (df["rooms"]
                     .str.split(" ")
                     .str[0]
                     .str.replace("--","0")
                     .astype(int))

In [14]:
# Exatamente igual rooms
set(df["bathrooms"])

{'-- Banheiro',
 '1 Banheiro',
 '2 Banheiros',
 '3 Banheiros',
 '4 Banheiros',
 '5 Banheiros',
 '6 Banheiros',
 '7 Banheiros',
 '8 Banheiros'}

In [15]:
df["bathrooms_limpo"] = (df["bathrooms"]
                         .str.split(" ")
                         .str[0]
                         .str.replace("--","0")
                         .astype(int))

In [16]:
# Exatamente igual rooms
set(df["garages"])

{'-- Vaga',
 '1 Vaga',
 '10 Vagas',
 '14 Vagas',
 '2 Vagas',
 '3 Vagas',
 '4 Vagas',
 '47 Vagas',
 '5 Vagas',
 '6 Vagas',
 '7 Vagas',
 '8 Vagas',
 '9 Vagas'}

In [17]:
df["garages_limpo"] = (df["garages"]
                       .str.split(" ")
                       .str[0]
                       .str.replace("--","0")
                       .astype(int))

In [18]:
set(df["garages_limpo"])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 47}

In [19]:
set(df["price"])

{'R$ 801.671',
 'R$ 384.568',
 'R$ 409.150',
 'R$ 554.141',
 'R$ 1.006.000',
 'R$ 211.900',
 'R$ 511.607',
 'R$ 1.234.431',
 'R$ 336.000',
 'R$ 1.535.200',
 'R$ 416.000',
 'R$ 998.076',
 'R$ 394.796',
 'R$ 3.500.017',
 'R$ 1.234.500',
 'R$ 1.566.822',
 'R$ 1.536.800',
 'R$ 1.319.000',
 'R$ 461.400',
 'R$ 2.787.000',
 'R$ 457.900',
 'R$ 1.385.000',
 'R$ 1.093.000',
 'R$ 976.000',
 'R$ 1.070.200',
 'R$ 404.000',
 'R$ 1.300.000',
 'R$ 354.930',
 'R$ 1.697.050',
 'R$ 1.597.000',
 'R$ 1.334.700',
 'R$ 1.843.476',
 'R$ 2.574.970',
 'R$ 334.000',
 'R$ 418.015',
 'R$ 1.560.000',
 'R$ 824.200',
 'R$ 1.200.000',
 'R$ 1.081.000',
 'R$ 368.000',
 'R$ 944.801',
 'R$ 688.000',
 'R$ 1.427.414',
 'R$ 305.000',
 'R$ 972.682',
 'R$ 563.999',
 'R$ 1.437.000',
 'R$ 258.700',
 'R$ 210.340',
 'R$ 779.000',
 'R$ 547.710',
 'R$ 115.000',
 'R$ 504.599',
 'R$ 171.000',
 'R$ 1.209.130',
 'R$ 382.000',
 'R$ 512.496',
 'R$ 537.000',
 'R$ 541.549',
 'R$ 452.830',
 'R$ 1.046.000',
 'R$ 498.300',
 'R$ 305.139',
 'R$ 

In [20]:
# Fazemos o split no "R$ ", pegamos o segundo elemento, 
# substituimos o . por nada e transformamos em int
# Ex: "R$ 824.200".split("R$ ") = ["R$ ", "824.200"]
# segundo elemento "824.200"
# "824.200".replace(".", "") = "824200"
# int("824200") = 824200 (passando de string para int)

df["price_limpo"] = [int(w.split("R$ ")[1].replace(".","")) for w in df["price"]]

In [21]:
# Passamos os na para a string "MISSING"
df["condo"] = df["condo"].fillna("MISSING")

In [22]:
# Fazemos semelhante ao price mas com a condicional de apenas para os que não não "MISSING".
# Os que são missing vamos substituir por np.nan
df["condo_limpo"] = [int(w.split("R$ ")[1].replace(".","")) if w!="MISSING" else np.nan for w in df["condo"]]

In [23]:
# Area já está limpo, é só passar para in
df["area_limpo"] = df["area"].astype(int)

In [24]:
# Crawled at já está limpo, é só passar para datetime
df["crawled_at"] = pd.to_datetime(df["crawled_at"], format="%Y-%m-%d %H:%M")

In [25]:
# Removendo colunas antigas
df = df.drop(columns=["area", "rooms", "bathrooms", "garages", "price", "condo"])

# Bairro

In [26]:
df["address"]

0         Avenida Portugal, 401 - Brooklin, São Paulo - SP
1            Rua Nova York, 470 - Brooklin, São Paulo - SP
2         Avenida Portugal, 371 - Brooklin, São Paulo - SP
3                  Rua Michigan - Brooklin, São Paulo - SP
4                                 Brooklin, São Paulo - SP
                               ...                        
15519                 Rua Purus - Tucuruvi, São Paulo - SP
15520      Rua Manuel Gaya, 945 - Tucuruvi, São Paulo - SP
15521    Avenida Coronel Sezefredo Fagundes - Tucuruvi,...
15522    Avenida Coronel Sezefredo Fagundes - Tucuruvi,...
15523       Avenida Mazzei, 530 - Tucuruvi, São Paulo - SP
Name: address, Length: 15524, dtype: object

In [27]:
# Vamos fazer um split no "- " e pegar o segundo elemento. 
# Depois um split na "," e pegar o primeiro elemento
# Exemplo: "Rua Purus - Tucuruvi, São Paulo - SP".split("- ") = ["Rua Purus", "Tucuruvi, Sao Paulo", "SP]
# Pegamos o segundo elemento: "Tucuruvi, São Paulo"
# "Tucuruvi, Sao Paulo".split(",") = ["Tucuruvi", "São Paulo"]
# Pegamos o primeiro elemento: "Tucuruvi"
df["bairro_teste"] = (df["address"]
                      .str.split("- ")
                      .str[1]
                      .str.split(",")
                      .str[0])

In [28]:
# Vamos refazer com outra estratégia apenas nos casos onde o processo anterior 
# ao inves de bairro nos deu "SP" e "RJ".
# Para esses casos, simplesmente split na "," e pegamos o primeiro elemento
# Ex: "Brooklin, São Paulo - SP".split(",") = ["Brooklin", "São Paulo"]
# Primeiro elemento "Brooklin"
df.loc[df["bairro_teste"].isin(["SP", "RJ"]), "bairro_teste"] = (df
                                                                 .loc[df["bairro_teste"].isin(["SP", "RJ"]), "address"]
                                                                 .str.split(",").str[0])

In [29]:
df.loc[df["crawler"]!=df["bairro_teste"],"bairro_teste"].unique()

array(['Butantã', 'República', 'Brás', 'Freguesia do Ó', 'Higienópolis',
       'Móoca', 'Saúde', 'Campo Limpo'], dtype=object)

In [30]:
# Passando o conteúdo de "crawler" para minúscula e substituindo " " por "_"
df["crawler"] = (df["crawler"]
                 .str.lower()
                 .str.replace(" ","_"))

In [31]:
# Passando o conteúdo de "bairro_test" para minúscula, 
# substituindo " " por "_" e removendo acentos
df["bairro_teste"] = (df["bairro_teste"]
                      .str.lower()
                      .str.replace(" ","_")
                      .str.normalize("NFKD")
                      .str.encode("ascii", errors="ignore")
                      .str.decode("utf-8"))

In [32]:
# Verificamos que freguesia estava escrito errado
df.loc[df["crawler"]!=df["bairro_teste"], ["crawler","bairro_teste"]]

,crawler,bairro_teste
3596,fregesia_do_o,freguesia_do_o
3597,fregesia_do_o,freguesia_do_o
3598,fregesia_do_o,freguesia_do_o
3599,fregesia_do_o,freguesia_do_o
3600,fregesia_do_o,freguesia_do_o
...,...,...
4521,fregesia_do_o,freguesia_do_o
4522,fregesia_do_o,freguesia_do_o
4523,fregesia_do_o,freguesia_do_o
4524,fregesia_do_o,freguesia_do_o


In [33]:
# Corrigimos e tudo bem
df["crawler"] = (df["crawler"].str.replace("fregesia", "freguesia"))

In [34]:
df.loc[df["crawler"]!=df["bairro_teste"], ["crawler","bairro_teste"]]

,crawler,bairro_teste


In [35]:
df = df.drop(columns="bairro_teste")

# Amenities

In [36]:
# Verificamos que o conteúdo da coluna amenities é uma string
df["amenities"]

0        Elevador\nAr-condicionado\nPlayground\nMóvel p...
1        Piscina\nElevador\nCondomínio fechado\nEspaço ...
2                             Academia\nCondomínio fechado
3        Piscina\nVaranda gourmet\nMobiliado\nChurrasqu...
4        Piscina\nVaranda gourmet\nChurrasqueira\nVaran...
                               ...                        
15519          Playground\nSalão de festas\nSalão de jogos
15520    Piscina\nVaranda gourmet\nChurrasqueira\nVaran...
15521                                          Aquecimento
15522                                          Aquecimento
15523    Piscina\nMobiliado\nChurrasqueira\nAcademia\nE...
Name: amenities, Length: 15524, dtype: object

In [37]:
# Transforma amenities em lista dando o split no caracter \n
df["amenities"] = df["amenities"].str.split("\n")

In [38]:
# Veja que virou uma lista
df["amenities"].head()

0    [Elevador, Ar-condicionado, Playground, Móvel ...
1    [Piscina, Elevador, Condomínio fechado, Espaço...
2                       [Academia, Condomínio fechado]
3    [Piscina, Varanda gourmet, Mobiliado, Churrasq...
4    [Piscina, Varanda gourmet, Churrasqueira, Vara...
Name: amenities, dtype: object

In [39]:
# Preenchemos os dados missing por ""
df["amenities"] = df["amenities"].fillna("")

In [40]:
# Queremos saber quais os amenities que existem
# Vamos criar uma lista vazia e, para cada linha da columna "amenities"
# Se a linha não for vazia, junta a o conteudo da lista na nossa lista de amenities
amenities = []
for am in df["amenities"]:
    if am != "":
        amenities = amenities + am

In [41]:
# Usamos o set para pegar apenas os items únicos
unique_amenities = list(set(amenities))

In [42]:
# A coluna amenities ainda é uma lista, vamos passá-la de volta a string
# para podermos usar a função contains. Para voltar a string, usarmos 
# A função join
df["amenities"].head()

0    [Elevador, Ar-condicionado, Playground, Móvel ...
1    [Piscina, Elevador, Condomínio fechado, Espaço...
2                       [Academia, Condomínio fechado]
3    [Piscina, Varanda gourmet, Mobiliado, Churrasq...
4    [Piscina, Varanda gourmet, Churrasqueira, Vara...
Name: amenities, dtype: object

In [43]:
# Vamos juntar cada item com uma ", "
df["amenities"] = df["amenities"].str.join(", ")

In [44]:
# Verificando apenas como ficou
df["amenities"].head()

0    Elevador, Ar-condicionado, Playground, Móvel p...
1    Piscina, Elevador, Condomínio fechado, Espaço ...
2                         Academia, Condomínio fechado
3    Piscina, Varanda gourmet, Mobiliado, Churrasqu...
4    Piscina, Varanda gourmet, Churrasqueira, Varan...
Name: amenities, dtype: object

In [45]:
# Agora, para cada amenity único, vamos criar uma coluna com o nome da amenity
# Vamos iniciála com 0
# Vamos procurar as linhas cuja coluna amenities contem a nossa amenity
# Nessas linhas, na coluna recem criada amenity, vamos substituir o 0 por 1
for amenity in unique_amenities:
    print(amenity)
    df[amenity]=0
    df.loc[df["amenities"].str.contains(amenity), amenity] = 1

Aceita animais
Área de serviço
Estacionamento para visitantes
Jardim
Ar-condicionado
Salão de jogos
Playground
Segurança 24h
Conexão à internet
Portão eletrônico
Quadra poliesportiva
Isolamento térmico
Churrasqueira
Garagem
Acesso para deficientes
Vista para o mar
Academia
Cozinha americana
Gramado
Espaço gourmet
Portaria 24h
Piscina para adulto
Varanda gourmet
Próximo a transporte público
Piscina infantil
Cabeamento estruturado
Spa
Closet
Depósito
Condomínio fechado
Cozinha
Armário embutido
Circuito de segurança
Churrasqueira na varanda
Serviços públicos essenciais
TV a cabo
Cinema
Vigia
Piso laminado
Pista de cooper
Gerador elétrico
Espaço verde / Parque
Sala de massagem
Lareira
Perto de vias de acesso
Sala pequena
Área de lazer
Elevador
Quadra de tênis
Isolamento acústico
Piscina
Serviços pay per use
Mais de um andar
Sala de almoço
Sala de jantar
Sistema de alarme
Varanda
Vista para a montanha
Lavanderia
Bicicletário
Gás Encanado
Aquecimento
Interfone
Copa
Recepção
Cozinha grande
Mo

In [46]:
# Vamos ver o resultado
df.head()

,id,url,header,address,amenities,crawler,crawled_at,rooms_limpo,bathrooms_limpo,garages_limpo,...,Espelhos d'água,Móvel planejado,Armário na cozinha,Próximo a escola,Vista exterior,Próximo a shopping,Quadra de squash,Quintal,Sauna,Próximo a hospitais
0,2472650173,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos à Venda, 64m²","Avenida Portugal, 401 - Brooklin, São Paulo - SP","Elevador, Ar-condicionado, Playground, Móvel p...",brooklin,2021-03-11 01:57:00,2,2,1,...,0,1,0,0,0,0,0,0,0,0
1,2463778941,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos à Venda, 83m²","Rua Nova York, 470 - Brooklin, São Paulo - SP","Piscina, Elevador, Condomínio fechado, Espaço ...",brooklin,2021-03-11 01:57:00,2,3,2,...,0,0,0,0,0,0,0,0,0,0
2,2505723836,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 3 Quartos à Venda, 75m²","Avenida Portugal, 371 - Brooklin, São Paulo - SP","Academia, Condomínio fechado",brooklin,2021-03-11 01:57:00,3,2,1,...,0,0,0,0,0,0,0,0,0,0
3,2497648375,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com Quarto à Venda, 47m²","Rua Michigan - Brooklin, São Paulo - SP","Piscina, Varanda gourmet, Mobiliado, Churrasqu...",brooklin,2021-03-11 01:57:00,1,2,1,...,0,0,0,0,0,0,0,0,0,0
4,2437733374,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 4 Quartos à Venda, 167m²","Brooklin, São Paulo - SP","Piscina, Varanda gourmet, Churrasqueira, Varan...",brooklin,2021-03-11 01:57:00,4,4,3,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# Vamos salvar o resultado no arquivo primary
df.to_csv("aptos_primary.csv", index=False)